In [ ]:
import os
#os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)


from functools import partial
import jax_sbgeom.coils as jsc
import jax_sbgeom.flux_surfaces as jsf

from dataclasses import dataclass
import functools
from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh, cumulative_trapezoid_uniform_periodic, interp1d_jax
import StellBlanket.SBGeom as sbg
import StellBlanket

import matplotlib.pyplot as plt
vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[2]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

In [ ]:
fs_base = jsb.flux_surfaces.FluxSurface.from_hdf5(vmec_file)

In [ ]:

coilsd = onp.loadtxt("filament_s4uu_5ci_23")[:,:]
amax = 101
i = 0
coils = []
currents = []
for i in range(40):
#    if i not in [0, 4, 8, 12, 16, 20,24, 28, 32,35, 36,39] :
    coils.append(coilsd[i * amax : i * amax + amax -1,:-1])
    currents.append(coilsd[i * amax, -1])



jax_coils = [jsc.DiscreteCoil.from_positions(jnp.array(coil)) for coil in coils]
jax_coilset = jsc.CoilSet.from_list(jax_coils)
jax_currents = jnp.array(currents)

sbg_coilset = sbg.Coil_Set([sbg.Discrete_Coil(coils[i]) for i in range(len(coils))])
coilset = jsc.fourier_coil.convert_to_fourier_coilset(jax_coilset)
coilset_resampled = jsc.fourier_coil.convert_fourier_coilset_to_equal_arclength(coilset)

#fs_coilset = jsc.FiniteSizeCoilSet.from_coilset(coilset_resampled, jsc.CentroidFrame)


## TODO 
#FIX Finite size coilset reverse
# Coilset optimization cws



In [ ]:
from StellBlanket.SBGeom import Coils_jax as CJ

cj_set = CJ.CoilSet_JAX(coilset_resampled.coils.centre_i, coilset_resampled.coils.fourier_cos, coilset_resampled.coils.fourier_sin)

In [ ]:
coilset_flipped = jsc.coilset.ensure_coilset_clockwise(coilset_resampled)

In [ ]:
positions = coilset_flipped.position(jnp.linspace(0,1,100))

In [ ]:
import pyvista as pv
plotter = pv.Plotter()
for i in range(coilset_flipped.n_coils):
    
    spline = pv.Spline(positions[i])
    plotter.add_mesh(spline, scalars = jnp.linspace(0,1,100), line_width=5, cmap="viridis")

plotter.show()